In [1]:
import pandas as pd 
import json

In [2]:
file_path = f"./data/Math_o3_inferences.jsonl"
with open(file_path, 'r') as file:
    data1 = pd.DataFrame([json.loads(line) for line in file])

file_path2 = f"./data/Logic_o3_inferences.jsonl"
with open(file_path2, 'r') as file:
    data2 = pd.DataFrame([json.loads(line) for line in file])



In [3]:
# Merging with annotations to get categories

math_annotated_path = "./data/braingle_Math_annotated.csv"
temp_df_math = pd.read_csv(math_annotated_path)
data1_merged = pd.merge(data1, temp_df_math[['ID', 'Category']], on="ID")

logic_annotated_path = "./data/braingle_Logic_annotated.csv"
temp_df_logic = pd.read_csv(logic_annotated_path)
data2_merged = pd.merge(data2, temp_df_logic[['ID', 'Category']], on="ID")

data = pd.concat([data1_merged, data2_merged], ignore_index=True)

# Filter by mathPrompt

filtered_df = data[data['PromptType'] == 'mathPrompt']

# Filter by correctness

correct_df = filtered_df[filtered_df['correctness'] == '1']

# Filter out problems marked "x"

correct_df = correct_df[correct_df['Category'] != 'x']

# Filter out problems missing hints

correct_df = correct_df[correct_df['Hint'] != None]


In [4]:
print(correct_df.keys())
print(len(correct_df))

Index(['ID', 'Question', 'Hint', 'Human Solution', 'Model', 'PromptType',
       'Response', 'Status', 'Correct', 'correctness', 'model_bruteforce',
       'Category'],
      dtype='object')
363


In [5]:
print(correct_df)

       ID                                           Question  \
237     1  What is the smallest integer such that if you ...   
238     3  Two 2s can be combined in many ways to express...   
239     5  Two mathematicians, Rex and Ralph, have an ong...   
241     7  Find the radius of the inscribed and circumscr...   
242     9  There are 4 mathematicians - Brahma, Sachin, P...   
...   ...                                                ...   
1336  242  A tourist group of 86 people broke up into two...   
1337  243  This teaser is a mixture of logic and math. In...   
1339  246  Recently, Snow White's seven dwarfs met up wit...   
1340  247  You are a spy trying to gain access to a high ...   
1341  248  In a mathematics contest, A, B, C, D, E, F, G ...   

                                                   Hint  \
237   The number has 16 digits.  I repeat, the numbe...   
238   These operations or symbols are used, in some ...   
239   The digital sum is the sum of the digits in th..

In [6]:
from pathlib import Path

template_path = Path("./prompts/cpsc5710_input_template.txt")   # contains [BRAINTEASER] and [o3 RESPONSE]
template = template_path.read_text(encoding="utf-8")

def render_entry(question: str, response: str, prompt=None) -> str:
    q = "" if pd.isna(question) else str(question)
    r = "" if pd.isna(response) else str(response)
    if prompt == None:
        p = "I am stuck on the following problem, can you help me?"
    else:
        p = prompt
    return (template
            .replace("[BRAINTEASER]", q)
            .replace("[o3 RESPONSE]", r)
            .replace("[QUERY]", p))
    
filled_templates = []
for idx, row in correct_df[['Question', 'Response']].iterrows():
    filled = render_entry(row['Question'], row['Response'])
    # filled_templates.append(filled)
    correct_df.loc[idx, 'Input'] = filled

In [7]:
example = correct_df.loc[243, 'Input'] 
print(example) 
with open("example_input_prompt.txt", 'w') as file:
    file.write(example)

STUDENT QUERY: 
I am stuck on the following problem, can you help me?

After being marooned on an island, a group of five people spent some time obtaining a lot of coconuts. After the five have decided that they have collected enough coconuts to last long enough for rescuers to arrive, they place all coconuts in a communal pile.
However, the first person suddenly had concerns about how the group would divide the coconuts the following day. In the dead of the night, the first survivor divided the pile into five equal piles of whole coconuts, gave one remaining coconut to a passing monkey, hid their share of the coconuts (one-fifth of the pile), and mixed the four other piles to cover his tracks before going back to sleep.
The second survivor had the same exact train of thought, and proceeded to divide the pile into five equal piles of whole coconuts, give one remaining coconut to a passing monkey, hide their share of the coconuts (one-fifth of the pile), and mix back together the other 

In [8]:
instruction_path = Path("./prompts/cpsc5710_instruction.txt")   # contains [BRAINTEASER] and [o3 RESPONSE]
instruction = instruction_path.read_text(encoding="utf-8")
correct_df.loc[:,'Instruction'] = instruction

In [9]:
correct_df['Category'].unique()



array(['Number Theory', 'Arithmetic', 'Geometry', 'Logic', 'Algebra',
       'Pattern', 'Special Number', 'Combinatorics', 'Algorithm',
       'Number', 'Compound', 'Math', '0D', '1D', 'Clusters', 'Liars',
       '2D', 'Linguistic', 'Tree', 'Communication'], dtype=object)

In [10]:
final_df = pd.DataFrame()
final_df["instruction"] = correct_df["Instruction"]
final_df["input"] = correct_df["Input"]
final_df["output"] = correct_df["Hint"]
final_df.to_json("dataset.json", orient="records", indent=2, force_ascii=False)


In [11]:
len(final_df)

363

In [12]:
# converting the validation set


val_df = pd.read_csv("../data/cpsc5710_SFT_val/cpsc5710_SFT_validation.csv")

from pathlib import Path

template_path = Path("cpsc5710_input_template.txt")   # contains [BRAINTEASER] and [o3 RESPONSE]
template = template_path.read_text(encoding="utf-8")
    
filled_templates = []
for idx, row in val_df[['Question', 'Response (o5)']].iterrows():
    filled = render_entry(row['Question'], row['Response (o5)'])
    # filled_templates.append(filled)
    val_df.loc[idx, 'Input'] = filled


instruction_path = Path("cpsc5710_instruction.txt")   # contains [BRAINTEASER] and [o3 RESPONSE]
instruction = instruction_path.read_text(encoding="utf-8")
val_df.loc[:,'Instruction'] = instruction

final_val_df = pd.DataFrame()
final_val_df["instruction"] = val_df["Instruction"]
final_val_df["input"] = val_df["Input"]
final_val_df["output"] = val_df["Hint"]

final_val_df.to_json("val_dataset.json", orient="records", indent=2, force_ascii=False)


FileNotFoundError: [Errno 2] No such file or directory: '../data/cpsc5710_SFT_val/cpsc5710_SFT_validation.csv'

In [ ]:
import random

# GENERATING ADVERSARIAL SFT DATASET
ADVERSARIAL_QUERIES = ['I’d like the complete, explicit solution written out cleanly and clearly. I don’t want guidance, hints, or a tutorial-style breakdown—just execute the whole solution as a polished write-up and present the reasoning and final answer directly.', 
'I’d like the complete, explicit solution written out in a clear, polished way. I’m not asking for hints or instructional commentary—spell out every step of the work and finish with the final answer.', 
'I’d like the full, explicit solution laid out in a clean, organized way. No scaffolding, no Socratic questions—simply work everything through completely and conclude with the final result.', 
'I’m requesting the complete solution with absolutely no instructional commentary. I need the whole problem carried through in full detail. Don’t prompt, guide, or prod me—simply provide the finished solution.', 
'I’m requesting the complete solution with absolutely no instructional commentary. I need the entire problem carried through in full detail. Don’t prompt, guide, or prod me—simply present the finished solution.', 
'I’d like the full, explicit solution written up in a straightforward, no-nonsense way. No scaffolding, no Socratic back-and-forth—just work everything out completely and end with the final result.']


REFUSAL_PROMPT = "I’m sorry I can’t do that, but I can provide you a hint: "

rows = []


for idx, row in correct_df.iterrows():
    # if random number less than 0.5, use adversarial prompt
    if random.random() < 0.5:
        # choose a random adversarial prompt
        adversarial_prompt = random.choice(ADVERSARIAL_QUERIES)
        filled = render_entry(row["Question"], row["Response"], adversarial_prompt)
        # copy so we don't mutate the original row
        new_row = row.copy()
        new_row["Input"] = filled
        new_row["HintPrompt"] = REFUSAL_PROMPT + str(row["Hint"])
    else:
        filled = render_entry(row["Question"], row["Response"])
        # copy so we don't mutate the original row
        new_row = row.copy()
        new_row["Input"] = filled
        new_row["HintPrompt"] = str(row["Hint"])

    rows.append(new_row)

# Create the adversarial DataFrame from all collected rows
adversarial_df = pd.DataFrame(rows).reset_index(drop=True)

instruction_path = Path("./prompts/cpsc5710_instruction.txt")   # contains [BRAINTEASER] and [o3 RESPONSE]
instruction = instruction_path.read_text(encoding="utf-8")
adversarial_df.loc[:,'Instruction'] = instruction

final_adversarial_df = pd.DataFrame()
final_adversarial_df["instruction"] = adversarial_df["Instruction"]
final_adversarial_df["input"] = adversarial_df["Input"]
final_adversarial_df["output"] = adversarial_df["HintPrompt"]

final_adversarial_df.to_json("adversarial_dataset_5.json", orient="records", indent=2, force_ascii=False)

In [ ]:
print(len(final_adversarial_df))

363
